In [1]:
import pandas as pd
import pysal
import numpy as np
#from pysal.lib import weights

In [2]:
data_filepath = '/cluster/home/maikents/coral-mapping/processed_data/df_ready_for_training.parquet'
df = pd.read_parquet(data_filepath)

FileNotFoundError: [Errno 2] No such file or directory: '/cluster/home/maikents/coral-mapping/processed_data/df_ready_for_training.parquet'

In [54]:
coral_df = df[['x', 'y', 'label']]
coral_df = coral_df.dropna()

In [55]:
absence_df = coral_df[coral_df['label'] == 0]
downsampled_absences = absence_df.sample(n=718, random_state=42)


In [57]:
presence_df = coral_df[coral_df['label'] == 1]

balanced_df = pd.concat([downsampled_absences, presence_df])

# Shuffle the dataframe
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [58]:
labels = balanced_df['label'].values.reshape(-1, 1)


In [59]:
from esda.moran import Moran

In [60]:
coords = balanced_df[['x', 'y']].values

In [65]:


w = weights.KNN.from_array(coords, k=4)  # Create a KNN spatial weights matrix based on the coords
                                         # k=4 means that the matrix should include the 4 nearest
                                         # neighbors for each data point.

/cluster/home/maikents/PyEnvCoralMapping/lib/python3.12/site-packages/libpysal/weights/distance.py:153: UserWarning: The weights matrix is not fully connected: 
 There are 4 disconnected components.
  W.__init__(self, neighbors, id_order=ids, **kwargs)


In [66]:
w.transform = 'r'

In [67]:

# Use the 'label' column as the variable of interest
moran = Moran(labels, w)

# Print Moran's I and p-value
print(f"Moran's I: {moran.I}")
print(f"p-value: {moran.p_sim}")

Moran's I: 0.6814066852367688
p-value: 0.001


In [64]:
#Moran's I: nan
#p-value: 0.001
print(f"Labels shape: {labels.shape}")
print(f"Weights shape: {w.sparse.shape}")


Labels shape: (1436, 1)
Weights shape: (78900000, 78900000)
